In [14]:
import os
import pandas as pd
import numpy as np
os.getcwd()

'c:\\Users\\HP\\Dropbox\\maaz iba work\\Iba\\semester 5\\Intro to Data Management\\Challenge 2\\property-price-predicting'

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [16]:
#data setting
training =pd.read_csv("Training.csv")
testing =pd.read_csv("Testing.csv")

In [18]:
#replace missing values with mean
training.fillna(training.mean(),inplace=True)
testing.fillna(testing.mean(),inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_14716\3174432764.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  training.fillna(training.mean(),inplace=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_14716\3174432764.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  testing.fillna(testing.mean(),inplace=True)


In [20]:
#one hot encoding for training data
stringcol =training.select_dtypes(include=['object']).columns
onehotencodedtraining=pd.get_dummies(training,columns=stringcol)

In [21]:
#one hot encoding for testing data
stringcol =testing.select_dtypes(include=['object']).columns
onehotencodedtesting=pd.get_dummies(testing,columns=stringcol)

In [22]:
#correlation check
print(onehotencodedtesting.shape)
print(onehotencodedtraining.shape)
correl=onehotencodedtraining.corrwith(onehotencodedtraining['price_doc'])
print(abs(correl))
for col in onehotencodedtraining.columns:
    if correl[col]<0.04:
        onehotencodedtraining.drop(col,axis=1,inplace=True)
        onehotencodedtesting.drop(col,axis=1,inplace=True,errors='ignore')
print(onehotencodedtesting.shape)
print(onehotencodedtraining.shape)


(30000, 231)
(100000, 232)
ID                      0.085433
floor                   0.186730
raion_popul             0.157995
green_zone_part         0.014515
indust_part             0.082620
                          ...   
ecology_excellent       0.037278
ecology_good            0.001036
ecology_no data         0.241627
ecology_poor            0.117835
ecology_satisfactory    0.064450
Length: 232, dtype: float64
(30000, 94)
(100000, 95)


In [23]:
#using correlation matrix to drop values
cor_matrix = onehotencodedtraining.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
#print(upper_tri)
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.4) ]
print(to_drop)
onehotencodedtraining.drop(to_drop,axis=1,inplace=True)
onehotencodedtesting.drop(to_drop,axis=1,inplace=True,errors='ignore')
print(onehotencodedtesting.shape)
print(onehotencodedtraining.shape)

['preschool_quota', 'preschool_education_centers_raion', 'school_quota', 'healthcare_centers_raion', 'sport_objects_raion', 'shopping_centers_raion', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_brick', 'build_count_panel', 'build_count_before_1920', 'build_count_1946-1970', 'build_count_1971-1995', 'ID_railroad_terminal', 'trc_count_500', 'sport_count_500', 'trc_count_1000', 'trc_sqm_1000', 'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000', 'sport_count_1000', 'market_count_1000', 'office_count_1500', 'trc_count_1500', 'trc_sqm_1500', 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500', 'sport_count_1500', 'market_count_1500', 'trc_sqm_2000', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'trc_count_3000', 'trc_sqm_3000', 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'sport_count_3000', 'market_count_3000', 'prom_part_5000', 'trc_sqm_5000', 'cafe

C:\Users\HP\AppData\Local\Temp\ipykernel_14716\4203410449.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))


In [9]:
#DROPPING CATEGORIAL VALS IF NO ONE HOT ENCODING
training=training.drop(stringcol,axis=1)
training=training.drop(['ID'],axis=1)
testing=testing.drop(['ID'],axis=1)
testing=testing.drop(stringcol,axis=1)

In [24]:
x_train=training.drop(columns=['price_doc'])
y_train=training['price_doc']

In [25]:
#setting x and y for training data
x=onehotencodedtraining.drop(columns=['price_doc'])
y=onehotencodedtraining['price_doc']
print(x.shape)
print(y.shape)

(100000, 40)
(100000,)


In [29]:
regressor = RandomForestRegressor(max_depth=25,max_features=20,n_estimators=700, random_state=25)
regressor.fit(x,y)


RandomForestRegressor(max_depth=25, max_features=20, n_estimators=700,
                      random_state=25)

In [30]:
y_pred = regressor.predict(onehotencodedtesting)

In [28]:
submission=pd.read_csv("sample.csv")
submission['price_doc']=y_pred
submission.to_csv("sample.csv",index=False)